# carrefour_search_trends.py

Fetches weekly search volume index (SVI) data for Carrefour-related keywords in France using the SearchAPI.io endpoint for Google Trends.

### This script will:

1. Load keywords and their category from an Excel file.
2. Batch keywords in groups of 5 to respect the API  request limit.
3. Query SearchAPI.io for weekly SVI per keyword (from 2022-01-01 to 2025-05-21).
4. Apply a retry mechanism for rate-limiting (429) errors.
5. Securely use API key via a .env file.
6. Save outputs:
   - carrefour_search_trends_keywords.csv (wide format)
   - carrefour_search_trends_aggregated.csv (by category)

In [ ]:
#install libraries
!pip install requests

In [ ]:
#import libraries
import os
import time
import requests
import pandas as pd
from datetime import datetime

In [1]:
#configurations
EXCEL_PATH = "Carrefour-related_keywords.xlsx"
START_DATE = "2022-01-01"
END_DATE = "2025-05-21"
GEO = "FR"
API_KEY = ""
API_URL = "https://www.searchapi.io/api/v1/search"
MAX_RETRIES = 2
BATCH_SIZE = 5

In [ ]:
#load selected keywords from excel
df_keywords = pd.read_excel("Carrefour-related_keywords.xlsx")
df_keywords.columns = df_keywords.columns.str.strip()
df_keywords = df_keywords.dropna(subset=["Keywords"])

In [ ]:
print(df_keywords.columns)  # Check actual column names
print(df_keywords.head())   # Inspect the first few rows

In [ ]:
#ensure the aggregation is kept
categories = df_keywords.groupby("Aggregate")
all_weeks = pd.date_range(start=START_DATE, end=END_DATE, freq='W-MON')
wide_df = pd.DataFrame(index=all_weeks)

In [ ]:
#fetch SVI for up to 5 keywords (Search.io API limit)
def fetch_keywords_trend(keywords):
    params = {
        "engine": "google_trends",
        "q": ",".join(keywords),
        "geo": GEO,
        "data_type": "TIMESERIES",
        "time": f"{START_DATE} {END_DATE}",
        "tz": "60",  #time zone: CET(UTC+1)
        "api_key": API_KEY
    }

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = requests.get(API_URL, params=params)
            response.raise_for_status()
            data = response.json()

            timeline = data.get("interest_over_time", {}).get("timeline_data", [])
            if not timeline:
                print(f"No timeline data for batch: {keywords}")
                return pd.DataFrame()

            records = []
            for entry in timeline:
                date = pd.to_datetime(entry["timestamp"], unit='s')
                for value in entry["values"]:
                    records.append({
                        "date": date,
                        "keyword": value["query"],
                        "value": int(value["extracted_value"])
                    })

            df = pd.DataFrame(records)
            wide_df = df.pivot(index="date", columns="keyword", values="value").sort_index()
            return wide_df

        except Exception as e:
            print(f"Error fetching batch {keywords} (attempt {attempt}): {e}")
            time.sleep(2 ** attempt)

    return pd.DataFrame()


In [ ]:
#merge all data collected
final_df = pd.DataFrame()

In [ ]:
# Fetch data in batches
all_keywords = df_keywords["Keywords"].tolist()
for i in range(0, len(all_keywords), BATCH_SIZE):
    batch = all_keywords[i:i + BATCH_SIZE]
    print(f"Fetching batch: {batch}")
    batch_df = fetch_keywords_trend(batch)
    final_df = pd.concat([final_df, batch_df], axis=1)

In [ ]:
#save keyword data
final_df = final_df.loc[:, ~final_df.columns.duplicated()].sort_index()
final_df.to_csv("carrefour_search_trends_keywords.csv")

In [ ]:
#aggregate data by category
df_keywords.set_index("Keywords", inplace=True)
aggregated_df = pd.DataFrame(index=final_df.index)

for category, keywords in df_keywords.groupby("Aggregate"):
    keyword_list = keywords.index.intersection(final_df.columns)
    aggregated_df[category] = final_df[keyword_list].sum(axis=1)

In [ ]:
#save aggregated keyword data
aggregated_df.to_csv("carrefour_search_trends_aggregated.csv")